In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
import random
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN

from nltk.corpus import stopwords
import nltk

from sklearn.feature_extraction.text import CountVectorizer

e:\deep-learning-on-practice-NLP\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yxtag\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
df_cv = pd.read_csv("../data/datasets_with_text/cv.csv")
df_vac = pd.read_csv("../data/datasets_with_text/vac.csv")

In [29]:
cvs = df_cv["cv_text"]
vacs = df_vac["vac_text"]

Обучим модель для поиска резюме по вакансии

In [36]:
umap_model = UMAP(n_neighbors=20, n_components=384, metric='cosine', low_memory=False, random_state=42)
vectorizer_model = CountVectorizer(stop_words=stopwords.words('russian') + stopwords.words('english'), ngram_range=(1, 2))
hdbscan_model = HDBSCAN(min_cluster_size=2, min_samples=1, metric='euclidean', prediction_data=True)

cv_topic_model = BERTopic(umap_model=umap_model,
                       vectorizer_model=vectorizer_model,
                       hdbscan_model=hdbscan_model,
                       top_n_words=10,
                       nr_topics=500,
                       language='multilingual', verbose=True)

topics, probs = cv_topic_model.fit_transform(cvs)

2023-12-23 02:39:56,480 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 1399/1399 [01:53<00:00, 12.29it/s]
2023-12-23 02:42:02,692 - BERTopic - Embedding - Completed ✓
2023-12-23 02:42:02,692 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2023-12-23 02:45:23,304 - BERTopic - Dimensionality - Completed ✓
2023-12-23 02:45:23,358 - BERTopic - Cluster - Start clustering the reduced embeddings
2023-12-23 02:50:24,361 - BERTopic - Cluster - Completed ✓
2023-12-23 02:50:24,362 - BERTopic - Representation - Extracting topics from clusters using representation models.
2023-12-23 02:51:42,108 - BERTopic - Representation - Completed ✓
2023-12-23 02:51:42,209 - BERTopic - Topic reduction - Reducing number of topics
2023-12-23 02:52:55,394 - BERTopic - Topic reduction - Reduced number of topics from 10766 to 500


In [39]:
cvs = cvs.to_frame()
cvs["topic_id"] = topics
cvs["topic_prob"] = probs

In [37]:
cv_topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,12141,-1_работы_месяцев_разработка_интернет,"[работы, месяцев, разработка, интернет, ru, оп...","[Пол, возраст:\nМужчина , 40 лет , родился 29..."
1,0,4264,0_москва_разработка_управление_проектов,"[москва, разработка, управление, проектов, раб...","[Пол, возраст:\nМужчина , 35 лет , родился 11..."
2,1,2285,1_готов_график_должность_инженер,"[готов, график, должность, инженер, занятость,...","[Пол, возраст:\nМужчина , 32 года , родился 1..."
3,2,2246,2_работы_оборудования_отдела_ит,"[работы, оборудования, отдела, ит, готов, ооо,...","[Пол, возраст:\nМужчина , 41 год , родился 5 ..."
4,3,1982,3_москва_готов_должность_командировки москва,"[москва, готов, должность, командировки москва...","[Пол, возраст:\nМужчина , 34 года , родился 2..."
...,...,...,...,...,...
495,494,2,494_android_соблюдение принципов_android прило...,"[android, соблюдение принципов, android прилож...","[Пол, возраст:\nМужчина , 27 лет , родился 9 ..."
496,495,2,495_https_com https_ilmixgroup ru_ilmixgroup,"[https, com https, ilmixgroup ru, ilmixgroup, ...","[Пол, возраст:\nМужчина , 32 года , родился 1..."
497,496,2,496_оператор_гипрочные кровельные_toyota bosho...,"[оператор, гипрочные кровельные, toyota boshok...","[Пол, возраст:\nМужчина , 34 года , родился 3..."
498,497,2,497_hoff_самара_hoff самара_достижение,"[hoff, самара, hoff самара, достижение, gprs о...","[Пол, возраст:\nМужчина , 26 лет , родился 3 ..."


In [40]:
cvs[cvs["topic_id"] == -1]

,cv_text,topic_id,topic_prob
3,"Пол, возраст:\nМужчина , 38 лет , родился 25 ...",-1,0.0
13,"Пол, возраст:\nЖенщина , 27 лет , родилась 15...",-1,0.0
18,"Пол, возраст:\nМужчина , 30 лет , родился 6 а...",-1,0.0
19,"Пол, возраст:\nМужчина , 38 лет , родился 23 ...",-1,0.0
20,"Пол, возраст:\nМужчина , 31 год , родился 30 ...",-1,0.0
...,...,...,...
44725,"Пол, возраст:\nМужчина , 68 лет , родился 10 ...",-1,0.0
44728,"Пол, возраст:\nМужчина , 30 лет , родился 24 ...",-1,0.0
44730,"Пол, возраст:\nЖенщина , 23 года , родилась 2...",-1,0.0
44731,"Пол, возраст:\nМужчина , 27 лет , родился 16 ...",-1,0.0


In [41]:
print(cvs[cvs["topic_id"] == -1].iloc[6]["cv_text"])

Пол, возраст:
Женщина ,  27 лет , родилась 17 января 1992
ЗП:
20000 руб.
Ищет работу на должность::
секретарь,администратор,помощник руководителя.
Город, переезд, командировки:
Зеленодольск (Республика Татарстан) , не готова к переезду , готова к редким командировкам
Занятость:
полная занятость
График:
полный день
Опыт работы:
Опыт работы 9 лет 3 месяца  Март 2010 — по настоящее время 9 лет 3 месяца Ангам,Универмаг Зум. Республика Татарстан Продавец-консультант Консультирование покупателей в торговом зале,выкладка товара на витрины,работа за кассовым аппаратом,ревизия,учёт,работа в программе 1С.
Последнее/нынешнее место работы:
Ангам,Универмаг Зум.
Последняя/нынешняя должность:
Продавец-консультант
Образование и ВУЗ:
Высшее образование 2012 судостроительный техникум. Эксплуатация средств связи., Связист-техник 2012 Казанский национальный исследовательский технологический университет, Казань Институт дополнительного профессионального образования
Авто:
Не указано



In [42]:
cv_topic_model.save("topic_model_for_cv", serialization="safetensors", save_ctfidf=True)

Теперь обучим модель для поиска вакансии по cv

In [48]:
umap_model = UMAP(n_neighbors=20, n_components=384, metric='cosine', low_memory=False, random_state=42)
vectorizer_model = CountVectorizer(stop_words=stopwords.words('russian') + stopwords.words('english'), ngram_range=(1, 2))
hdbscan_model = HDBSCAN(min_cluster_size=2, min_samples=1, metric='euclidean', prediction_data=True)

vac_topic_model = BERTopic(umap_model=umap_model,
                       vectorizer_model=vectorizer_model,
                       hdbscan_model=hdbscan_model,
                       top_n_words=10,
                       nr_topics=500,
                       language='multilingual', verbose=True)

topics, probs = vac_topic_model.fit_transform(vacs)

2023-12-23 03:24:55,748 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 977/977 [00:32<00:00, 29.97it/s]
2023-12-23 03:25:32,323 - BERTopic - Embedding - Completed ✓
2023-12-23 03:25:32,323 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2023-12-23 03:27:36,751 - BERTopic - Dimensionality - Completed ✓
2023-12-23 03:27:36,787 - BERTopic - Cluster - Start clustering the reduced embeddings
2023-12-23 03:29:42,736 - BERTopic - Cluster - Completed ✓
2023-12-23 03:29:42,736 - BERTopic - Representation - Extracting topics from clusters using representation models.
2023-12-23 03:30:04,841 - BERTopic - Representation - Completed ✓
2023-12-23 03:30:04,862 - BERTopic - Topic reduction - Reducing number of topics
2023-12-23 03:30:26,955 - BERTopic - Topic reduction - Reduced number of topics from 7238 to 500


In [49]:
vac_topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,8396,-1_опыт_работы_требования_знание,"[опыт, работы, требования, знание, опыт работы...",[Сфера деятельности:\nПродажи\nДолжность:\nМен...
1,0,2290,0_продажи_продаж_клиентов_клиентами,"[продажи, продаж, клиентов, клиентами, деятель...",[Сфера деятельности:\nПродажи\nДолжность:\nПро...
2,1,1195,1_тестирования_тестирование_тест_qa,"[тестирования, тестирование, тест, qa, тестиро...",[Сфера деятельности:\nТестирование\nДолжность:...
3,2,976,2_проектами_управление проектами_управление_пр...,"[проектами, управление проектами, управление, ...",[Сфера деятельности:\nУправление проектами\nДо...
4,3,941,3_инженер_оборудования_деятельности инженер_ин...,"[инженер, оборудования, деятельности инженер, ...",[Сфера деятельности:\nУправление проектами\nДо...
...,...,...,...,...,...
495,494,2,494_проекта устанавливать_разработка монетизац...,"[проекта устанавливать, разработка монетизацио...",[Сфера деятельности:\nУправление проектами\nДо...
496,495,2,495_разработки серверного_qa заниматься_разраб...,"[разработки серверного, qa заниматься, разрабо...","[Сфера деятельности:\nПрограммирование, Разраб..."
497,496,2,496_инфраструктуры клиентов_2008r2_cisco_asterisk,"[инфраструктуры клиентов, 2008r2, cisco, aster...","[Сфера деятельности:\nКонсалтинг, Аутсорсинг\n..."
498,497,2,497_high_scalability_spring_etc,"[high, scalability, spring, etc, good understa...","[Сфера деятельности:\nПрограммирование, Разраб..."


In [45]:
vac_topic_model.save("topic_model_for_vac", serialization="safetensors", save_ctfidf=True)